# NCC-PINN - Automated Experiments Runner

This notebook runs automated architecture search experiments for NCC-PINN on Google Colab GPU.

**Features:**
- Mounts Google Drive for persistent storage
- Clones the NCC-PINN repository from GitHub
- Installs dependencies with CUDA support
- Runs multiple experiments sequentially with different architectures
- Auto-generates datasets if missing
- Auto-saves all outputs (plots, metrics, checkpoints) to Drive
- Generates comparison report across all experiments

**Runtime:** GPU (T4 or better recommended)

---


## 1. Check GPU Availability


In [ ]:
# Check if GPU is available
!nvidia-smi


## 2. Mount Google Drive

All outputs will be saved to: `/content/drive/MyDrive/NCC-PINN/`


In [ ]:
from google.colab import drive
import os

# Mount Drive
drive.mount('/content/drive')

# Create output directory on Drive
DRIVE_OUTPUT_DIR = '/content/drive/MyDrive/NCC-PINN'
os.makedirs(DRIVE_OUTPUT_DIR, exist_ok=True)

print(f"\n✓ Drive mounted. Outputs will be saved to: {DRIVE_OUTPUT_DIR}")


## 3. Clone Repository


In [ ]:
# Configuration
REPO_URL = "https://github.com/assafzimand/NCC-PINN.git"
REPO_NAME = "NCC-PINN"
WORK_DIR = f"/content/{REPO_NAME}"

# Clone or update repository
if os.path.exists(WORK_DIR):
    print(f"Repository already exists at {WORK_DIR}")
    print("Pulling latest changes...")
    !cd {WORK_DIR} && git pull
else:
    print(f"Cloning repository from {REPO_URL}...")
    !git clone {REPO_URL} {WORK_DIR}

# Change to repo directory
os.chdir(WORK_DIR)
print(f"\n✓ Working directory: {os.getcwd()}")


## 4. Install Dependencies


In [ ]:
# Install PyTorch with CUDA for Colab (CUDA 12.1)
print("Installing PyTorch with CUDA support...")
%pip install torch torchvision --index-url https://download.pytorch.org/whl/cu121 --quiet

print("\nInstalling project dependencies...")
%pip install -r requirements.txt --quiet

# Verify CUDA is available
import torch
print(f"\n✓ PyTorch version: {torch.__version__}")
print(f"✓ CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✓ CUDA version: {torch.version.cuda}")
    print(f"✓ GPU device: {torch.cuda.get_device_name(0)}")
else:
    print("⚠ WARNING: CUDA not available! Training will be slow on CPU.")


## 5. (Optional) View Experiment Plan

View the current experiment configuration before running.


In [ ]:
# Display current experiment plan
!cat experiments_plan.yaml


## 6. Run Experiments & Auto-Save to Drive

**This cell does everything:**
1. Generates datasets if they don't exist
2. Runs all experiments sequentially
3. For each experiment: trains model, runs NCC analysis (periodic + final)
4. Generates comparison report across all experiments
5. Saves all results to Google Drive

**Note:** This may take several hours depending on number of experiments and GPU type.


In [ ]:
import shutil
from pathlib import Path
from datetime import datetime

print("="*70)
print("NCC-PINN: Automated Experiments")
print("="*70)

# Run experiments (unbuffered for real-time output)
!python -u run_experiments.py

print("\n" + "="*70)
print("Experiments Complete! Saving to Google Drive...")
print("="*70)

# Find the experiments directory
experiments_dir = Path("outputs/experiments")
exp_folders = sorted(experiments_dir.glob("*/"), key=lambda x: x.stat().st_mtime)

if exp_folders:
    latest_exp = exp_folders[-1]
    
    # Copy to Drive
    dest = Path(DRIVE_OUTPUT_DIR) / latest_exp.name
    shutil.copytree(latest_exp, dest, dirs_exist_ok=True)
    
    print(f"\nSaved to: {dest}")
    print(f"\nExperiment Directory Structure:")
    print(f"  - experiments_plan.yaml (configuration used)")
    print(f"  - comparison_summary.csv (metrics comparison table)")
    print(f"  - training_and_results_comparison.png (training/eval metrics)")
    print(f"  - ncc_classification_comparison.png (NCC accuracy per layer/epoch)")
    print(f"  - ncc_compactness_comparison.png (NCC margin per layer/epoch)")
    print(f"  - schrodinger_layers-X-Y-Z_act-tanh/ (individual model results)")
    print(f"    - <timestamp>/")
    print(f"      - training_plots/ (curves, predictions)")
    print(f"      - ncc_plots/ (final + periodic epochs)")
    print(f"      - schrodinger_heatmaps.png")
    print(f"      - schrodinger_fixed_time.png")
    print(f"      - schrodinger_uv_components.png")
    print(f"      - metrics.json, summary.txt, config_used.yaml")
    
    print(f"\n{'='*70}")
    print(f"All experiments saved to Google Drive!")
    print(f"{'='*70}")
else:
    print("No experiment folders found. Check for errors above.")


## 7. Notes & Tips

---

### Experiment Configuration
- Edit `experiments_plan.yaml` to customize:
  - Base configuration (problem, epochs, learning rate, etc.)
  - List of architectures to test
  - NCC evaluation frequency (`ncc_eval_every`)

### Experiment Structure
- **Base Config**: Common parameters for all experiments
- **Experiments List**: Each experiment specifies its name and architecture
- **Sequential Execution**: Experiments run one after another
- **Automatic Comparison**: Summary table and plots generated at the end

### Outputs
All results are automatically saved to Google Drive:

- **Parent Folder**: `MyDrive/NCC-PINN/<name>_<params>_<timestamp>/`
  - `experiments_plan.yaml`: Configuration used
  - `comparison_summary.csv`: Metrics comparison table
  - `comparison_plots.png`: Visual comparison
  
- **Per Experiment**: `<problem>_layers-<arch>_act-<activation>/`
  - `<timestamp>/`
    - `config_used.yaml`: Configuration for this run
    - `metrics.json`: Training metrics
    - `summary.txt`: Training summary
    - `training_plots/`: Training curves and predictions
    - `ncc_plots/`: Final NCC analysis
      - `ncc_plots_epoch_X/`: Periodic NCC analysis (if ncc_eval_every > 0)

### Tips
- **Use GPU runtime**: Runtime → Change runtime type → GPU (T4 recommended)
- **Long runs**: Use Colab Pro for extended sessions (experiments can take hours)
- **Results persist**: All results are auto-saved to Drive with timestamp
- **Monitor progress**: Check the output logs to track experiment progress
- **Compare architectures**: Download results from Drive to analyze locally

---
